# Verifying derivative calculations

This notebook describes the verification of derivative calculations using Taylor remainder convergence testing. A simple time-independent problem is considered, using tlm_adjoint with the [Firedrake](https://firedrakeproject.org/) backend.

The Taylor remainder convergence testing method is described in:

- P.&nbsp;E.&nbsp;Farrell, D. A. Ham, S. W. Funke, and M. E. Rognes, 'Automated derivation of the adjoint of high-level transient finite element programs', SIAM Journal on Scientific Computing 35(4), pp. C369&ndash;C393, 2013, doi: 10.1137/120873558

## Forward problem

We consider the solution of a linear time-independent partial differential equation, followed by the calculation of the square of the $L^2$-norm of the solution. Non-linearity is introduced by defining the right-hand-side of the problem to be a non-linear function of the control. We assume real spaces and a real build of Firedrake throughout.

Specifically we consider the solution $u \in V$ of

$$
  \forall \zeta \in V \qquad \int_\Omega u \zeta + \alpha^2 \int_\Omega \nabla u \cdot \nabla \zeta = \int_\Omega \left( \sin^2 m \right) \zeta,
$$

where $V$ is a real $P_1$ continuous finite element space defining functions on the domain $\Omega = \left( 0, 1 \right)^2$, with $m \in V$. This corresponds to a discretization of the partial differential equation

$$
    u - \alpha^2 \nabla^2 u = \sin^2 m \quad \text{on } \left( x, y \right) \in \left( 0, 1 \right)^2,
$$

subject to boundary conditions $\nabla u \cdot \hat{n} = 0$ on the boundary, where $\hat{n}$ is an outward unit normal.

A simple implementation in Firedrake, with $m = e^x \sin \left( \pi x y \right)$ and $\alpha = 0.2$, takes the form:

In [ ]:
from firedrake import *

mesh = UnitSquareMesh(50, 50)
X = SpatialCoordinate(mesh)
space = FunctionSpace(mesh, "Lagrange", 1)
test = TestFunction(space)
trial = TrialFunction(space)

m = Function(space, name="m")
m.interpolate(exp(X[0]) * sin(pi * X[0] * X[1]))

alpha = Constant(0.2)

u = Function(space, name="u")
solve(inner(trial, test) * dx + (alpha ** 2) * inner(grad(trial), grad(test)) * dx
      == inner(sin(m) ** 2, test) * dx, u)

J = assemble(inner(u, u) * dx)

## Taylor remainder convergence testing: First order

If we have a functional $J$ depending on a control $m$ then we have, given some perturbation direction $\zeta$, via Taylor expansion,

$$
  \left| J \left( m + \varepsilon \zeta \right) - J \left( m \right) \right| = O \left( \varepsilon \right), \\
$$
$$
  \left| J \left( m + \varepsilon \zeta \right) - J \left( m \right) - \varepsilon \frac{dJ}{dm} \zeta \right| = O \left( \varepsilon^2 \right).
$$

That is, $\zeta$ is some direction in the same space as $m$, which we choose, and then we control the perturbation amplitude using the scalar $\varepsilon$. The final term in the second absolute value is a directional derivative, which we can compute using the adjoint.

This leads to a methodology for verifying a derivative computed using the adjoint method:

1. Choose a direction $\zeta$.
2. Choose a number of different values of $\varepsilon$.
3. See if we have second order convergence of the second of the above, to zero.

This verifies only the directional derivative with a single direction, but if we wish we can choose a new direction and repeat the test.

We can use the `taylor_test` function to perform the test for us. By default this generates a pseudorandom direction and chooses a number of values of $\varepsilon$. It then computes the quantities on the left-hand-sides of the above equations, computes the orders of convergence between consecutive pairs of values for $\varepsilon$, and displays the results. It returns the *minimum* order computed for the second case, which in a successful verification should be close to two.

Let's compute a derivative using the adjoint method, and apply a Taylor remainder convergence test:

In [ ]:
from firedrake import *
from tlm_adjoint.firedrake import *

import logging
import numpy as np

np.random.seed(33582866)

logger = logging.getLogger("tlm_adjoint")
logger.setLevel(logging.INFO)
root_logger = logging.getLogger()
if len(logger.handlers) == 1:
    if len(root_logger.handlers) == 1:
        root_logger.handlers.pop()
    root_logger.addHandler(logger.handlers.pop())

reset_manager()

mesh = UnitSquareMesh(50, 50)
X = SpatialCoordinate(mesh)
space = FunctionSpace(mesh, "Lagrange", 1)
test = TestFunction(space)
trial = TrialFunction(space)

m = Function(space, name="m")
m.interpolate(exp(X[0]) * sin(pi * X[0] * X[1]))

alpha = Constant(0.2)


def forward(m):
    u = Function(space, name="u")
    solve(inner(trial, test) * dx + (alpha ** 2) * inner(grad(trial), grad(test)) * dx
          == inner(sin(m) ** 2, test) * dx, u)

    J = Functional(name="J")
    J.assign(inner(u, u) * dx)
    return J


start_manager()
J = forward(m)
stop_manager()

dJ_dm = compute_gradient(J, m)

min_order = taylor_test(forward, m, J_val=J.value, dJ=dJ_dm, seed=1.0e-3)
assert min_order > 1.99

The key changes here are:

- To define a `forward` function. `taylor_test` uses this to repeatedly rerun the forward with different values of $\varepsilon$.
- Using `seed` to control the considered values of $\varepsilon$. If this is too large then the asymptotic orders of convergence may not be seen. If this is too small then the effect of roundoff or iterative solver tolerances may become too large.
- Seeding the NumPy pseudorandom number generator. The pseudorandom direction is generated using `numpy.random.random`, and we seed the pseudorandom number generator to improve reproducibility. We could alternatively supply a direction using the `dM` argument.

We see the expected first and second orders of convergence.

## Taylor remainder convergence testing: Second order

Including the next order term in the Taylor expansion leads to

$$
    \left| J \left( m + \varepsilon \zeta \right) - J \left( m \right) - \varepsilon \frac{dJ}{dm} \zeta - \frac{1}{2} \varepsilon^2 \left[ \frac{d}{dm} \left( \frac{dJ}{dm} \zeta \right) \right] \zeta \right| = O \left( \varepsilon^3 \right).
$$

Let's use this approach to test Hessian calculations using a `CachedHessian`:

In [ ]:
np.random.seed(19986557)

H = CachedHessian(J)

min_order = taylor_test(forward, m, J_val=J.value, ddJ=H, seed=1.0e-3)
assert min_order > 2.99

We now see the expected first and *third* orders of convergence, although there is a suggestion of roundoff affecting the results for the smallest $\varepsilon$. Here the first order directional derivative is computed using a tangent-linear calculation, and the Hessian action on $\zeta$ is computed by applying the adjoint method to the forward and tangent-linear calculations.

## Taylor remainder convergence testing: Higher order

We can test the derivative of a directional derivative, if we substitute

$$
  J \rightarrow K = \frac{dJ}{dm} \zeta_0,
$$

with some *new* direction $\zeta_0$, which we choose. That is, we use

$$
  \left| K \left( m + \varepsilon \zeta \right) - K \left( m \right) \right| = O \left( \varepsilon \right), \\
$$
$$
  \left| K \left( m + \varepsilon \zeta \right) - K \left( m \right) - \varepsilon \frac{dK}{dm} \zeta \right| = O \left( \varepsilon^2 \right),
$$

with

$$
  K = \frac{dJ}{dm} \zeta_0.
$$

The new term

$$
  \frac{dK}{dm} \zeta
$$

can be computed using either a higher order tangent-linear or higher-order adjoint calculation. This generalizes naturally to higher order, by replacing the functional with the directional derivative of a directional derivative.

The function `taylor_test_tlm` performs such verification tests, considering directional derivatives of a given order, and computing all derivatives using tangent-linear calculations. Each directional derivative requires a new direction to be chosen &ndash; by default pseudorandom directions are generated.

Let's apply this test up to fourth order:

In [ ]:
np.random.seed(76149511)

for order in range(1, 5):
    min_order = taylor_test_tlm(forward, m, tlm_order=order, seed=1.0e-3)
    assert min_order > 1.99

The function `taylor_test_tlm_adjoint` also performs such verification tests, but computes the highest order derivative information using the adjoint method.

Let's apply this test up to fourth order:

In [ ]:
np.random.seed(74728054)

for order in range(1, 5):
    min_order = taylor_test_tlm_adjoint(forward, m, adjoint_order=order, seed=1.0e-3)
    assert min_order > 1.99